### Reading text data

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
         .appName("text-processing")
         .master("spark://spark-master:7077")
         .config("spark.executor.memory", "512m")
         .getOrCreate())

spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/05/19 12:11:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/05/19 12:11:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = (spark.read.format("csv")
      .option("header",True)
      .load("../data/Reviews.csv"))

In [5]:
df.printSchema()

root
 |-- Id: string (nullable = true)
 |-- ProductId: string (nullable = true)
 |-- UserId: string (nullable = true)
 |-- ProfileName: string (nullable = true)
 |-- HelpfulnessNumerator: string (nullable = true)
 |-- HelpfulnessDenominator: string (nullable = true)
 |-- Score: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- Summary: string (nullable = true)
 |-- Text: string (nullable = true)



In [6]:
df.show(10, truncate=False)

+------+----------+--------------+----------------------------------+--------------------+----------------------+-----+----------+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [7]:
# Apply regular expression to remove all non-alphabetic characters
df_clean = (df
            .withColumn("Text", regexp_replace("Text", "[^a-zA-Z ]", ""))
            .withColumn("Text", regexp_replace("Text", "  +", " ")))

df_clean.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|                     0|    4|1310083200|one of my favorit...|These cookies are...|
|250132|B005UBH8WC| AY12DBB0U420B|       Gary Peterson|                   0|                     0|    5|1336348800|A Favorite of My ...|Ive been using Ic...|
|250137|B001EQ57QG| A9X62UCTFNQBE|            

In [8]:
df_with_words = (df_clean.withColumn("words", split(df_clean.Text, "\\s+")))
df_with_words.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[I, was, interest...|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|                     0|    4|1310083200|one of my favorit...|These cookies are...|[These, cookies, ...|
|250132|B005UBH8WC| AY12DBB0U420B|       Gary Peterson|                   0|                     0| 

In [9]:
from pyspark.ml.feature import Tokenizer

# Tokenize the text data
tokenizer = Tokenizer(inputCol='Text', outputCol='words')
df_with_words = tokenizer.transform(df_clean)
df_with_words.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[i, was, interest...|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|                     0|    4|1310083200|one of my favorit...|These cookies are...|[these, cookies, ...|
|250132|B005UBH8WC| AY12DBB0U420B|       Gary Peterson|                   0|                     0| 

In [10]:
from pyspark.ml.feature import StopWordsRemover

remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df_stop_words_removed = remover.transform(df_with_words)

df_stop_words_removed.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|      filtered_words|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[i, was, interest...|[interested, food...|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|                     0|    4|1310083200|one of my favorit...|These cookies are...|[these, cookies, ...|[cookies, yummy, 

In [11]:
df_exploded = (df_stop_words_removed
               .select(explode(df_stop_words_removed.filtered_words).alias("word")))
word_count = (df_exploded
              .groupBy("word")
              .count()
              .orderBy("count", ascending=False))
word_count.show(n=100)

+---------+-----+
|     word|count|
+---------+-----+
|     like|11014|
|       br|10705|
|     good| 8717|
|    great| 7475|
|      one| 7471|
|    taste| 7435|
|   coffee| 7257|
|  product| 6593|
|   flavor| 6468|
|     love| 5968|
|      tea| 5947|
|     food| 5514|
|      get| 4675|
|   really| 4488|
|     much| 4109|
|     dont| 4054|
|     also| 3757|
|     time| 3716|
|   little| 3710|
|      use| 3695|
|   amazon| 3626|
|    tried| 3483|
|     best| 3434|
|      buy| 3381|
|     find| 3338|
|    price| 3324|
|      ive| 3317|
|       im| 3267|
|     even| 3191|
|     make| 3163|
|     well| 3113|
|      try| 2991|
|      dog| 2990|
|   better| 2985|
|      eat| 2899|
|    first| 2662|
|chocolate| 2522|
|    water| 2495|
|    found| 2492|
|     used| 2437|
|      bag| 2399|
|   bought| 2352|
|    sweet| 2282|
|      cup| 2236|
|    drink| 2212|
|     made| 2197|
|    sugar| 2155|
|      two| 2152|
|      box| 2139|
|    think| 2077|
|   tastes| 2053|
|      way| 2047|
|    since

In [12]:
from pyspark.ml.feature import CountVectorizer

# Convert the text data into numerical features
vectorizer = CountVectorizer(inputCol='filtered_words', outputCol='features')
vectorized_data = vectorizer.fit(df_stop_words_removed).transform(df_stop_words_removed)
vectorized_data.show(10, truncate=False)

+------+----------+--------------+----------------------------------+--------------------+----------------------+-----+----------+---------------------------------------------------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [13]:
(vectorized_data.repartition(1)
 .write.mode("overwrite")
 .json("../data/data_lake/reviews_vectorized.json"))

### Using the `regexp_extract()` function

In [14]:
from pyspark.sql.functions import regexp_extract

# Extract all words starting with "q"
df_q_words = (vectorized_data
              .withColumn("q_words", regexp_extract("text", "\\\\bq\\\\w*", 0)))
df_q_words.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|      filtered_words|            features|q_words|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[i, was, interest...|[interested, food...|(40697,[1,2,11,12...|       |
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|    

### Using the `rlike()` function

In [15]:
# Check if text data contains the word "good"
df_good_word = (vectorized_data
              .withColumn("contains_qood", expr("text rlike 'quick'")))
df_good_word.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|      filtered_words|            features|contains_qood|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[i, was, interest...|[interested, food...|(40697,[1,2,11,12...|        false|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...| 

### Customizing Stop Words

In [16]:
custom_stopwords = ["/><br", "-", "/>I","/>The"]
stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", stopWords=custom_stopwords)

df_stop_words_removed = stopwords_remover.transform(df_with_words)

df_stop_words_removed.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|      filtered_words|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[i, was, interest...|[i, was, interest...|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|                     0|    4|1310083200|one of my favorit...|These cookies are...|[these, cookies, ...|[these, cookies, 

In [17]:
custom_stopwords = ["/><br", "-", "/>I","/>The"]

stopwords_remover = StopWordsRemover(inputCol="words", outputCol="filtered_words", stopWords=custom_stopwords)
stopwords_remover.setStopWords(custom_stopwords)

df_stop_words_removed = stopwords_remover.transform(df_with_words)

df_stop_words_removed.show()

+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|    Id| ProductId|        UserId|         ProfileName|HelpfulnessNumerator|HelpfulnessDenominator|Score|      Time|             Summary|                Text|               words|      filtered_words|
+------+----------+--------------+--------------------+--------------------+----------------------+-----+----------+--------------------+--------------------+--------------------+--------------------+
|250093|B0029NII3C|A3P8CU9874SRK5|        C. christine|                   0|                     0|    2|1316649600|Unwanted Ingredients|I was interested ...|[i, was, interest...|[i, was, interest...|
|250115|B0013MEB40| A99TG4Q2ZPW7S|"Blu-estLight ""M...|                   0|                     0|    4|1310083200|one of my favorit...|These cookies are...|[these, cookies, ...|[these, cookies, 

In [18]:
spark.stop()